In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [7]:
df = pd.read_csv('WORD2VEC.csv')
df.head()

,TARGET,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,0,0.003003,-0.003955,-0.004124,0.006777,0.006404,-0.001013,-0.001003,0.001001,0.001729,...,-0.005132,-0.004970,0.003045,0.003143,-0.002204,-0.007240,-0.005164,0.005413,0.004564,-0.006225
1,0,-0.007776,-0.001502,-0.005474,-0.000234,-0.001707,0.002485,-0.000504,0.000600,0.001937,...,0.001066,-0.002839,-0.007328,-0.002091,-0.001709,-0.001146,0.001958,0.000744,-0.002432,0.002326
2,0,0.006793,-0.007658,0.001889,-0.007330,-0.004990,-0.004757,0.000385,-0.003287,0.004401,...,0.003459,-0.001208,-0.007642,0.005018,0.003703,0.001772,-0.001705,0.005737,0.004874,0.001709
3,0,0.002281,-0.000288,0.000654,-0.001300,-0.004144,-0.001902,0.004286,-0.003939,-0.002677,...,-0.004255,0.001768,0.004207,-0.006092,0.000729,-0.002757,-0.002375,0.004251,0.004612,0.000453
4,0,-0.003799,-0.004566,0.006317,-0.004503,0.004294,-0.002494,0.007146,-0.007071,-0.005247,...,-0.006083,0.003231,-0.007659,-0.002953,-0.000559,-0.001681,-0.001622,-0.002704,0.007071,0.004966


In [8]:
y = df['TARGET']
x = df.drop('TARGET', axis=1) 
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [9]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.543956  0.088463   
1  XGBClassifier(base_score=None, booster=None, c...  0.521193  0.042517   
2      LGBMClassifier(max_depth=10, random_state=50)  0.524333  0.048700   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.531397  0.062828   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.507064  0.014329   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.087912   0.549470  0.488226  0.517041     0.599686     0.488226  
1  0.042386   0.522998  0.481947  0.501634     0.560440     0.481947  
2  0.048666   0.525285  0.505495  0.515200     0.543171     0.505495  
3  0.062794   0.530395  0.547881  0.538996     0.514914     0.547881  
4  0.014129   0.506057  0.590267  0.544928     0.423862     0.590267  
